This is my implementation of Jurgen Arias' original Tensorflow model. As far as I can tell, the biggest change is that he had reorganized his LibreSpeech files into male and female directories. He also manually downloaded his Librespeech files, but I am using the torchaudio dataset.

In [1]:
import os
import torch
import torchaudio

dataroot = os.path.expanduser("~")
librispeech_data = torchaudio.datasets.LIBRISPEECH(dataroot, download=True)

In [2]:
import pandas as pd
from pathlib import Path

filelist = Path(os.path.join(dataroot, 'LibriSpeech', 'train-clean-100')).rglob('*.flac')

files = [{'speaker_id': int(str(file)[len(dataroot)+1:].split('/')[2]), 'file': file} for file in filelist]
df_files=pd.DataFrame(files)

/tmp/ipykernel_1239/816679470.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
speakers_file = os.path.join(dataroot, "LibriSpeech", "SPEAKERS.TXT")
speakers = pd.read_table(
    speakers_file,
    engine='python',
    sep='\s+\|\s+',
    names=['id','gender','subset','duration','name'],
    dtype={'id': 'i','gender': 'U1', 'subset': 'U', 'duration': 'f', 'name': 'U'}, comment=';')
speakers = speakers.drop(speakers[
    (speakers['subset']=='train-clean-360')
    |(speakers['subset']=='train-other-500')
    |(speakers['subset']=='dev-other')
    |(speakers['subset']=='test-other')
].index)
speakers.head()

,id,gender,subset,duration,name
3,19,F,train-clean-100,25.190001,Kara Shallenberg
8,26,M,train-clean-100,25.080000,Denny Sayers
9,27,M,train-clean-100,20.139999,Sean McKinley
14,32,F,train-clean-100,24.010000,Betsie Bush
18,39,F,train-clean-100,25.049999,Sherry Crowther


In [4]:
# Although this function was modified and many parameteres were explored with, most of it
# came from Source 8 (sources in the READ.ME)
import librosa
import numpy as np


def extract_features(files):
    
    # Sets the name to be the path to where the file is in my computer
    file_name = str(files.file)

    # Loads the audio file as a floating point time series and assigns the default sample rate
    # Sample rate is set to 22050 by default
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 

    # Generate Mel-frequency cepstral coefficients (MFCCs) from a time series 
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)

    # Generates a Short-time Fourier transform (STFT) to use in the chroma_stft
    stft = np.abs(librosa.stft(X))

    # Computes a chromagram from a waveform or power spectrogram.
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)

    # Computes a mel-scaled spectrogram.
    mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)

    # Computes spectral contrast
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)

    # Computes the tonal centroid features (tonnetz)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    sr=sample_rate).T,axis=0)
        
    
    # We add also the classes of each file as a label at the end
    label = files.speaker_id

    return mfccs, chroma, mel, contrast, tonnetz, label

In [5]:
# The following operation can take hours depending on your cpu.
# Luckily, we only have to run it once, then we can save and load the processed data from a pickle file
import os
from datetime import datetime
from tqdm import tqdm

picklefile = 'Librispeech_features_label.pkl'
if os.path.isfile(picklefile):
    print(f"Reading features_label from {picklefile}")
    features_label = pd.read_pickle(picklefile)
else:
    tqdm.pandas()
    starttime = datetime.now()
    features_label = df_files.progress_apply(extract_features, axis=1)
    print(f"Extracting features took {datetime.now() - startTime}")
features_label.shape

Reading features_label from Librispeech_features_label.pkl


(28539,)

In [6]:
import numpy as np

features = []
for i in range(0, len(features_label)):
    features.append(np.concatenate((features_label[i][0], features_label[i][1], 
                features_label[i][2], features_label[i][3],
                features_label[i][4]), axis=0))
df_files['X'] = features

In [7]:
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()
labels = to_categorical(lb.fit_transform(df_files['speaker_id']))
df_files['y']=labels.tolist()

2024-04-14 18:58:56.158863: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [8]:
# Split samples into train/val/test
from fast_ml.model_development import train_valid_test_split

df_files["Sets"] = "Training"
for lbl in df_files['speaker_id'].unique():
    temp_data = df_files[df_files['speaker_id'] == lbl]
    X_train, y_train, X_val, y_val, X_test, y_test = train_valid_test_split(
        temp_data,
        target="speaker_id",
        train_size=0.6,
        valid_size=0.1,
        test_size=0.3
    )
    df_files.Sets.iloc[X_test.index] = "Testing"
    df_files.Sets.iloc[X_val.index] = "Validation"

In [9]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

In [10]:
ss.fit(np.array(df_files['X'].tolist()))

X_train = ss.transform(np.array(df_files[df_files['Sets']=='Training']['X'].tolist()))
y_train = np.array(df_files[df_files['Sets']=='Training']['y'].tolist())
X_val = ss.transform(np.array(df_files[df_files['Sets']=='Validation']['X'].tolist()))
y_val = np.array(df_files[df_files['Sets']=='Validation']['y'].tolist())
X_test = ss.transform(np.array(df_files[df_files['Sets']=='Testing']['X'].tolist()))
y_test = np.array(df_files[df_files['Sets']=='Testing']['y'].tolist())

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout 
from keras.callbacks import EarlyStopping


model = Sequential()

model.add(Dense(193, input_shape=(193,), activation="relu"))
model.add(Dropout(0.1))

model.add(Dense(128, activation="relu"))
model.add(Dropout(0.25))

model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))

model.add(Dense(y_train.shape[1], activation="softmax"))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=1, mode='auto')

2024-04-14 19:04:17.200751: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-04-14 19:04:17.201044: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-04-14 19:04:18.749661: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-14 19:04:18.750217: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:03:00.0 name: NVIDIA GeForce GTX 750 Ti computeCapability: 5.0
coreClock: 1.137GHz coreCount: 5 deviceMemorySize: 1.95GiB deviceMemoryBandwidth: 80.47GiB/s
2024-04-14 19:04:18.750337: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2024-04-14 19:04:18.803474: I tensorflow/stream_executor/platform/de

In [ ]:
start = datetime.now()
history = model.fit(
    X_train, y_train,
    batch_size=256,
    epochs=100, 
    validation_data=(X_val, y_val),
    callbacks=[early_stop]
)
print(f"{datetime.now()-start} seconds")

In [ ]:
import tensorflow as tf

input_signature = [tf.TensorSpec([1, 193], tf.float32, name='X')]

In [ ]:
import tf2onnx
import onnx

onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature)

In [ ]:
onnx.save(onnx_model, 'Librispeech_Tensorflow_model.onnx')

In [ ]:
import pickle

with open('Librispeech_Tensorflow_X_train.pkl','wb') as f:
    pickle.dump(X_train, f)
with open('Librispeech_Tensorflow_y_train.pkl','wb') as f:
    pickle.dump(y_train, f)
with open('Librispeech_Tensorflow_X_val.pkl','wb') as f:
    pickle.dump(X_val, f)
with open('Librispeech_Tensorflow_y_val.pkl','wb') as f:
    pickle.dump(y_val, f)
with open('Librispeech_Tensorflow_X_test.pkl','wb') as f:
    pickle.dump(X_test, f)
with open('Librispeech_Tensorflow_y_test.pkl','wb') as f:
    pickle.dump(y_test, f)

In [ ]:
model.summary()